# Распознавание именованных сущностей
*Исследовательский анализ данных*

## Датасет

Будем использовать подмножество из *Cross-lingual Transfer Evaluation of Multilingual Encoders (XTREME)*, называемый *WikiANN* или *PAN-X*.

Датасет содержит статьи из Википедии на многих языках, включая 4 наиболее распространенных языков в Швейцарии:
- *Немецкий (62.9%)*,
- *Французский (22.9%)*,
- *Итальянский (8.4%)*,
- *Английский (5.9%)*.

Статьи аннотированы тегами
- *LOC (location)*,
- *PER (person)*,
- *ORG (organization)*

в формате *inside-outside-beginning (IOB2)*. В этом формате
- начало токена сущности помечается с префиксом `B-`,
- токены, не являющиеся начальными, но входящие в состав именной сущности, помечаются префиксом `I-`.
- Токены, не имеющие отношение к сущности, помечаются `O`.

In [1]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME имеет {len(xtreme_subsets)} конфигураций")

/home/misha/.pyenv/versions/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


XTREME имеет 183 конфигураций


Нам нужны только *PAN-X* подмножества для наших 4-ех языков.

In [2]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:5]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de']

Чтобы создать реалистичный Швейцарский корпус, нам сделать так, чтобы количество образцов в языке соответствовало распределению языков в стране. В датасетах разных языков одинаковое количество образцов.

### Загрузка датасетов

In [3]:
from collections import defaultdict
from datasets import load_dataset, DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=42)
            .select(range(int(frac * ds[split].num_rows)))
        )

In [4]:
import pandas as pd

pd.DataFrame(
    {lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
    index=["Количество тренировочных образцов"],
)

,de,fr,it,en
Количество тренировочных образцов,12580,4580,1680,1180


### Обзор образцов

In [5]:
for k, v in panx_ch["de"]["train"][0].items():
    print(f"{k}: {v}")

tokens: ['Olympique', 'Nîmes', ',', 'Auxerres', 'seinerzeitiger', 'drittklassiger', 'Endspielgegner', ',', 'hatte', 'sich', 'erst', 'gar', 'nicht', 'für', 'die', 'Hauptrunde', 'qualifizieren', 'können', '.']
ner_tags: [3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [6]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])


In [7]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}


panx_de = panx_ch["de"].map(create_tag_names)

In [8]:
de_example = panx_de["train"][0]
pd.DataFrame(
    [de_example["tokens"], de_example["ner_tags_str"]], ["Tokens", "Tags"]
)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Tokens,Olympique,Nîmes,",",Auxerres,seinerzeitiger,drittklassiger,Endspielgegner,",",hatte,sich,erst,gar,nicht,für,die,Hauptrunde,qualifizieren,können,.
Tags,B-ORG,I-ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O


### Распределение тегов

In [9]:
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

,ORG,PER,LOC
train,5397,5881,6169
validation,2639,2870,3172
test,2657,2971,3100


### Модель

Напишем сами голову к `XML-RoBERTa` для классификации токенов, хоть и есть готовый класс для этого (`XLMRobertaForTokenClassification`).

In [10]:
import torch
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel


class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs,
    ):
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

## Загрузка модели

In [11]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [12]:
from transformers import AutoConfig

xlmr_model_name = "xlm-roberta-base"

xlmr_config = AutoConfig.from_pretrained(
    xlmr_model_name,
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index,
)

In [13]:
from transformers import AutoTokenizer


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

xlmr_model = XLMRobertaForTokenClassification.from_pretrained(
    xlmr_model_name, config=xlmr_config
).to(device)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
text = "Jack Sparrow loves New York!"
tokenized_text = xlmr_tokenizer(text, return_tensors="pt")
pd.DataFrame(
    [tokenized_text.tokens(), tokenized_text.input_ids[0].numpy()],
    index=["Tokens", "Input IDs"],
)

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,37456,15555,5161,7,2356,5753,38,2


In [15]:
outputs = xlmr_model(tokenized_text.input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)

print(f"Число токенов в последовательности {len(tokenized_text.tokens())}")
print(f"Размерность выхода {outputs.shape}")

Число токенов в последовательности 10
Размерность выхода torch.Size([1, 10, 7])


In [16]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([tokenized_text.tokens(), preds], index=["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Tags,I-PER,I-LOC,I-LOC,I-LOC,I-LOC,I-LOC,I-PER,I-PER,I-LOC,I-PER


## Токенизация

Датасет состоит из разбитых на слова токенов. Наш токенизатор разбивает эти слова на подслова. Мы можем классифицировать только начало слово, а остальную часть игнорировать. Это снизит вклад в функцию потерь длинных слов, которые состоят из нескольких токенов.

В этом поможет метод `tokenized_text.word_ids()`, которая возвращает индексы слов, из которых получены subtoken-ы.

In [17]:
words, labels = de_example["tokens"], de_example["ner_tags"]

In [18]:
tokenizer_input = xlmr_tokenizer(words, is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenizer_input["input_ids"])
word_ids = tokenizer_input.word_ids()

pd.DataFrame([tokens, word_ids], index=["Tokens", "Word IDs"])

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
Tokens,<s>,▁Olymp,ique,▁N,îm,es,▁,",",▁Au,xer,...,▁die,▁Haupt,runde,▁quali,fi,zieren,▁können,▁,.,</s>
Word IDs,None,0,0,1,1,1,2,2,3,3,...,14,15,15,16,16,16,17,18,18,None


`torch.nn.CrossEntropyLoss` по-умолчанию имеет `ignore_index=-100`. 

In [19]:
prev_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == prev_word_idx:
        label_ids.append(-100)
    elif word_idx != prev_word_idx:
        label_ids.append(labels[word_idx])
    prev_word_idx = word_idx

labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
Tokens,<s>,▁Olymp,ique,▁N,îm,es,▁,",",▁Au,xer,...,▁die,▁Haupt,runde,▁quali,fi,zieren,▁können,▁,.,</s>
Word IDs,None,0,0,1,1,1,2,2,3,3,...,14,15,15,16,16,16,17,18,18,None
Label IDs,-100,3,-100,4,-100,-100,0,-100,0,-100,...,0,0,-100,0,-100,-100,0,0,-100,-100
Labels,IGN,B-ORG,IGN,I-ORG,IGN,IGN,O,IGN,O,IGN,...,O,O,IGN,O,IGN,IGN,O,O,IGN,IGN


In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        prev_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == prev_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            prev_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
def encode_panx_dataset(corpus):
    return corpus.map(
        tokenize_and_align_labels,
        batched=True,
        remove_columns=["langs", "ner_tags", "tokens"],
    )

In [22]:
panx_ch["de"]["train"]

Dataset({
    features: ['tokens', 'ner_tags', 'langs'],
    num_rows: 12580
})

In [23]:
panx_de_encoded = encode_panx_dataset(panx_ch["de"])

Map: 100%|██████████| 6290/6290 [00:00<00:00, 32730.36 examples/s]


In [24]:
panx_de_encoded["train"].features

{'input_ids': List(Value('int32')),
 'attention_mask': List(Value('int8')),
 'labels': List(Value('int64'))}

## Метрика качества

Чтобы оценить модель для NER задачи используют обычные метрики для классификации (precision, recall, ...), но с некоторыми отличиями:
- нужно учитывать сущности, состоящие из несколько слов. Если хотя бы в одной части сущности модель ошиблась, то прогноз можно считать неверным;
- в подсчете метрик нельзя учитывать верно предсказанные токены, не являющиеся сущностями. Так как таких токенов больше всего, а нам важнее предсказать сущность, то эти предсказания вносят дисбаланс и метрика будет малоинформативна для нас.
С такой механикой работает библиотека `seqeval`.

In [25]:
import numpy as np
from seqeval.metrics import classification_report

y_true = [
    ["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],
    ["B-PER", "I-PER", "O"],
]

y_pred = [
    ["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"],
    ["B-PER", "I-PER", "O"],
]

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

        MISC       0.00      0.00      0.00         1
         PER       1.00      1.00      1.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



In [26]:
def align_predictions(predictions, label_ids):
    # predictions: (batch_size, seq_len, num_labels)
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # ignore index = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

## Обучение модели

[02.Multilingual-NER.Train.ipynb](02.Multilingual-NER.Train.ipynb)